<a href="https://colab.research.google.com/github/elviobg/titanic/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

import os
ROOT_PATH = '/content/drive/My Drive/Colab Notebooks/Titanic/'
DATASETS_PATH = '/content/drive/My Drive/Colab Notebooks/Titanic/dataset/'
os.chdir(ROOT_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd
import numpy as np

In [12]:
train_original = pd.read_csv(DATASETS_PATH+'train.csv')
test_original = pd.read_csv(DATASETS_PATH+'test.csv')
print("Data loaded...")
train_original.head()

Data loaded...


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
